In [1]:
import pandas as pd

In [16]:
binance_OHLCV = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/binance_OHLCV/binance_BTC_USDT_1h.csv')
blockstream = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/blockstream/btc_blockstream_blocks_since2020.csv')
binance_funding = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/derivatives_raw/binance_funding/binance_funding_BTCUSDT.csv')
binance_open_interest = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/derivatives_raw/binance_open_interest/binance_open_interest_BTCUSDT_1h.csv')
binance_taker_ls = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/derivatives_raw/binance_taker_longshort/binance_taker_longshort_BTCUSDT_1h.csv')
google_trends_bitcoin = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/google-trends/bitcoin/google_trends_bitcoin_daily.csv')
google_trends_bitcoin = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/google-trends/bitcoin/google_trends_bitcoin_daily.csv')
google_trends_btc = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/google-trends/btc/google_trends_btc_daily.csv')
google_trends_crypto_daily = pd.read_csv('/Users/danilokacanski/Desktop/master-rad-clean/2020/data/google-trends/crypto/google_trends_crypto_daily.csv')

In [17]:
binance_OHLCV.head()

,timestamp,open,high,low,close,volume,symbol,exchange
0,2020-01-01 00:00:00+00:00,7195.24,7196.25,7175.46,7177.02,511.814901,BTC/USDT,binance
1,2020-01-01 01:00:00+00:00,7176.47,7230.00,7175.71,7216.27,883.052603,BTC/USDT,binance
2,2020-01-01 02:00:00+00:00,7215.52,7244.87,7211.41,7242.85,655.156809,BTC/USDT,binance
3,2020-01-01 03:00:00+00:00,7242.66,7245.00,7220.00,7225.01,783.724867,BTC/USDT,binance
4,2020-01-01 04:00:00+00:00,7225.00,7230.00,7215.03,7217.27,467.812578,BTC/USDT,binance


In [18]:
blockstream.head()

,day,blocks_count,total_size_bytes,total_txs,avg_txs_per_block,avg_block_size
0,2020-01-01,174,111993256,251735,1446.752874,6.436394e+05
1,2020-01-02,141,142743474,295567,2096.219858,1.012365e+06
2,2020-01-03,169,153896396,316517,1872.881657,9.106296e+05
3,2020-01-04,168,133506838,287215,1709.613095,7.946836e+05
4,2020-01-05,163,128906654,287779,1765.515337,7.908384e+05


In [19]:
from pathlib import Path
# ---- 1) Make OHLCV the canonical hourly index ----
def to_utc(ts):
    ts = pd.to_datetime(ts, utc=True, errors="coerce")
    return ts

# Ensure OHLCV timestamps are UTC and on an hourly grid
ohlcv = binance_OHLCV.copy()
ohlcv["timestamp"] = to_utc(ohlcv["timestamp"])
ohlcv = ohlcv.sort_values("timestamp")

# Build a *complete* hourly index from min..max of OHLCV
hourly_index = pd.date_range(
    ohlcv["timestamp"].min().floor("H"),
    ohlcv["timestamp"].max().ceil("H"),
    freq="1H",
    tz="UTC",
)

# Start the merged frame with the hourly index, then put OHLCV on it
base = pd.DataFrame(index=hourly_index)
ohlcv_h = (
    ohlcv.set_index("timestamp")
         .sort_index()
         .reindex(hourly_index)             # keep gaps explicit
)
# (optional) small ffill on OHLCV if needed:
for c in ["open","high","low","close","volume","symbol","exchange"]:
    if c in ohlcv_h.columns:
        ohlcv_h[c] = ohlcv_h[c].ffill(limit=2)

merged = base.join(ohlcv_h, how="left")

# ---- 2) Helper upsamplers for other cadences ----
def upsample_daily_to_hourly(df, date_col, value_cols):
    s = df.copy()
    s[date_col] = pd.to_datetime(s[date_col], utc=True, errors="coerce").dt.normalize()
    s = s.dropna(subset=[date_col]).set_index(date_col).sort_index()
    # Make sure we have continuous daily; then carry forward to hourly
    s = s.asfreq("1D").ffill()
    return s.reindex(hourly_index).ffill()

def upsample_timestamp_to_hourly(df, ts_col, value_cols, ffill_limit=None):
    s = df.copy()
    s[ts_col] = to_utc(s[ts_col])
    s = s.dropna(subset=[ts_col]).set_index(ts_col).sort_index()
    # For already-hourly series, this is essentially an align+ffill; for subhourly/sparse, we ffill to hours.
    s = s[ value_cols ].reindex(hourly_index).ffill(limit=ffill_limit)
    return s

def upsample_every_8h_to_hourly(df, ts_col, value_cols):
    # Funding prints every 8h; allow ffill up to 8 hours between stamps
    return upsample_timestamp_to_hourly(df, ts_col, value_cols, ffill_limit=8)

# ---- 3) Blockstream (daily) → hourly ----
# Expected columns: 'day','blocks_count','total_size_bytes','total_txs','avg_txs_per_block','avg_block_size'
if not blockstream.empty:
    block_cols = [c for c in blockstream.columns if c != "day"]
    block_h = upsample_daily_to_hourly(blockstream, "day", block_cols)
    block_h.columns = [f"block_{c}" for c in block_h.columns]  # prefix to avoid name clashes
    merged = merged.join(block_h, how="left")

# ---- 4) Binance funding (8h) → hourly ----
# Expected: 'fundingTime','fundingRate','markPrice' (symbol is redundant for single market)
if "fundingTime" in binance_funding.columns:
    fund_cols = [c for c in ["fundingRate","markPrice"] if c in binance_funding.columns]
    if fund_cols:
        fund_h = upsample_every_8h_to_hourly(binance_funding, "fundingTime", fund_cols)
        fund_h.columns = [f"funding_{c}" for c in fund_h.columns]
        # convenient derived metric
        if "funding_fundingRate" in fund_h:
            fund_h["funding_annualized_pct"] = fund_h["funding_fundingRate"] * 3 * 365 * 100
        merged = merged.join(fund_h, how="left")

# ---- 5) Binance open interest (hourly) → hourly ----
# Expected: 'timestamp','sumOpenInterest','sumOpenInterestValue'
if not binance_open_interest.empty:
    oi_cols = [c for c in ["sumOpenInterest","sumOpenInterestValue"] if c in binance_open_interest.columns]
    if oi_cols:
        oi_h = upsample_timestamp_to_hourly(binance_open_interest, "timestamp", oi_cols, ffill_limit=3)
        oi_h["oi_change"] = oi_h["sumOpenInterest"].diff()
        oi_h["oi_value_change"] = oi_h["sumOpenInterestValue"].diff()
        merged = merged.join(oi_h.add_prefix("oi_"), how="left")

# ---- 6) Binance taker long/short ratio (hourly) → hourly ----
# Expected: 'timestamp','longShortRatio','longAccount','shortAccount'
if not binance_taker_ls.empty:
    tkr_cols = [c for c in ["longShortRatio","longAccount","shortAccount"] if c in binance_taker_ls.columns]
    if tkr_cols:
        tkr_h = upsample_timestamp_to_hourly(binance_taker_ls, "timestamp", tkr_cols, ffill_limit=3)
        # derive long/short shares if ratio present
        if "longShortRatio" in tkr_h.columns:
            r = tkr_h["longShortRatio"]
            with pd.option_context("mode.use_inf_as_na", True):
                tkr_h["long_share"]  = r / (1.0 + r)
                tkr_h["short_share"] = 1.0 - tkr_h["long_share"]
        merged = merged.join(tkr_h.add_prefix("tkr_"), how="left")

# ---- 7) Google Trends (daily) → hourly ----
def attach_trends(df_trends, default_col_name):
    if df_trends.empty:
        return
    df = df_trends.copy()
    # auto-detect the value column (first non-date)
    date_col = "date" if "date" in df.columns else df.columns[0]
    val_cols = [c for c in df.columns if c != date_col]
    if not val_cols:
        return
    # If the provider stored the column as e.g. 'Bitcoin' or 'btc', keep that; else use default_col_name
    name = val_cols[0] if len(val_cols)==1 else default_col_name
    df = df[[date_col, val_cols[0]]].rename(columns={val_cols[0]: name})
    tr_h = upsample_daily_to_hourly(df, date_col, [name])
    merged[name] = tr_h[name]

attach_trends(google_trends_bitcoin, "trends_bitcoin")
attach_trends(google_trends_btc,     "trends_btc")
attach_trends(google_trends_crypto_daily, "trends_crypto")

# ---- 8) Final tidy + save ----
merged.index.name = "timestamp"
merged = merged.sort_index()

# Optional: fill tiny gaps (few hours) for slow-moving fundamentals
for c in merged.columns:
    if merged[c].dtype != "O":  # non-object
        merged[c] = merged[c].ffill(limit=6)

out_path = "data/combined_hourly_dataset.csv"
Path(out_path).parent.mkdir(parents=True, exist_ok=True)
merged.to_csv(out_path)
print(f"✅ Saved combined dataset → {out_path}  (rows={len(merged)}, cols={merged.shape[1]})")
merged.head(3)

/var/folders/tz/yhx353f17tj_1s1dj9_gw3x80000gn/T/ipykernel_23108/995130449.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ohlcv["timestamp"].min().floor("H"),
/var/folders/tz/yhx353f17tj_1s1dj9_gw3x80000gn/T/ipykernel_23108/995130449.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ohlcv["timestamp"].max().ceil("H"),
/var/folders/tz/yhx353f17tj_1s1dj9_gw3x80000gn/T/ipykernel_23108/995130449.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_index = pd.date_range(
/var/folders/tz/yhx353f17tj_1s1dj9_gw3x80000gn/T/ipykernel_23108/995130449.py:94: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):


✅ Saved combined dataset → data/combined_hourly_dataset.csv  (rows=49363, cols=27)


,open,high,low,close,volume,symbol,exchange,block_blocks_count,block_total_size_bytes,block_total_txs,...,oi_oi_change,oi_oi_value_change,tkr_longShortRatio,tkr_longAccount,tkr_shortAccount,tkr_long_share,tkr_short_share,Bitcoin,BTC,crypto
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00+00:00,7195.24,7196.25,7175.46,7177.02,511.814901,BTC/USDT,binance,174.0,111993256.0,251735.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,26.0,33.0
2020-01-01 01:00:00+00:00,7176.47,7230.00,7175.71,7216.27,883.052603,BTC/USDT,binance,174.0,111993256.0,251735.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,26.0,33.0
2020-01-01 02:00:00+00:00,7215.52,7244.87,7211.41,7242.85,655.156809,BTC/USDT,binance,174.0,111993256.0,251735.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,26.0,33.0


In [21]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49363 entries, 2020-01-01 00:00:00+00:00 to 2025-08-18 18:00:00+00:00
Freq: h
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   open                     49363 non-null  float64
 1   high                     49363 non-null  float64
 2   low                      49363 non-null  float64
 3   close                    49363 non-null  float64
 4   volume                   49363 non-null  float64
 5   symbol                   49354 non-null  object 
 6   exchange                 49354 non-null  object 
 7   block_blocks_count       49363 non-null  float64
 8   block_total_size_bytes   49363 non-null  float64
 9   block_total_txs          49363 non-null  float64
 10  block_avg_txs_per_block  49363 non-null  float64
 11  block_avg_block_size     49363 non-null  float64
 12  funding_fundingRate      36100 non-null  float64
 13  funding_markPrice    

In [22]:
# Drop columns that have any missing values
cleaned = merged.dropna(axis=1, how="any")

print(f"Original shape: {merged.shape}")
print(f"Cleaned shape:  {cleaned.shape}")

cleaned.info()

Original shape: (49363, 27)
Cleaned shape:  (49363, 13)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49363 entries, 2020-01-01 00:00:00+00:00 to 2025-08-18 18:00:00+00:00
Freq: h
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   open                     49363 non-null  float64
 1   high                     49363 non-null  float64
 2   low                      49363 non-null  float64
 3   close                    49363 non-null  float64
 4   volume                   49363 non-null  float64
 5   block_blocks_count       49363 non-null  float64
 6   block_total_size_bytes   49363 non-null  float64
 7   block_total_txs          49363 non-null  float64
 8   block_avg_txs_per_block  49363 non-null  float64
 9   block_avg_block_size     49363 non-null  float64
 10  Bitcoin                  49363 non-null  float64
 11  BTC                      49363 non-null  float64
 12  crypto             

In [23]:
out_path = "data/combined_hourly_dataset.csv"
Path(out_path).parent.mkdir(parents=True, exist_ok=True)
cleaned.to_csv(out_path)